For this second sweep the following changes have been made:
 - Parameters have been reduced to the range around the optimal parameter found during sweep 1.
 - A new parameter called dataset is added that randomly chooses between strib and kolding datasets.
 - Some parameters have been defaulted such as optimizer and lookback.
     - This is done to improve the other parameters even more, as these parameters showed to be the best choice in run 1.
 - For the sake of experiment, I will add an option for zero additional layers to the number of layers parameter.
 - Seeds have been added.
 - Number of Epochs have been reduced.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import os
from utils import processing
from utils import utils

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, TimeDistributed, \
    BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import wandb
from wandb.keras import WandbCallback


In [2]:
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [3]:
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = (18,10)

In [4]:
wandb.login()

wandb: Currently logged in as: nbvanting (use `wandb login --relogin` to force relogin)


True

In [5]:
def load_data(config):
    # Load csv & parse dates to datetime index
    data = pd.read_csv(f'../data/processed/{config.dataset}_features.csv', index_col='Datetime', parse_dates=['Datetime'])
    # Select Features
    data = data[['Value', 'sunshine_mins', 'airtemp_c', 'daylength_hrs', 'wkdy_sin', 'wkdy_cos', 'wknd', 'mnth_sin', 'mnth_cos']]
    
    train, val, test = processing.create_datasets(data, split=split, 
                                                  steps=steps, lookback=config.lookback, 
                                                  horizon=horizon, batch_size=config.batch_size, 
                                                  scaler='standard')    
    return train, val, test
    
    
def build_model(config):
        
    model = Sequential()

    model.add(Input(shape=(config.lookback, config.num_features)))

    # CNN Block
    model.add(Conv1D(filters=config.cnn_layer_size_1, kernel_size=3, activation=config.activation_cnn))
    model.add(MaxPooling1D(pool_size=2))    
    for i in range(config.num_cnn_layers):
        model.add(Conv1D(filters=config.cnn_layer_size_2, kernel_size=3, activation=config.activation_cnn))
        model.add(MaxPooling1D(pool_size=2))
    
    # RNN Block
    for i in range(config.num_gru_layers):
        model.add(GRU(config.gru_layer_size_1, return_sequences=True, activation=config.activation_gru))
        model.add(Dropout(config.dropout))
    
    model.add(GRU(config.gru_layer_size_2, return_sequences=False, activation=config.activation_gru))
    model.add(Dropout(config.dropout))

    model.add(Dense(1))
    
    opt = config.optimizer
    if opt == 'sgd':
        opt = SGD(learning_rate=config.learning_rate, momentum=config.momentum)
    elif opt == 'rmsprop':
        opt = RMSprop(learning_rate=config.learning_rate)
    else:
        opt = Adam(learning_rate=config.learning_rate)
        
    model.compile(optimizer=opt, loss='mse', metrics=['mae', 'mape'])
    
    return model

def log_defaults():
    # Default values from the first sweep
    # The following values resulted in the strongest model
    wandb_config = {
        'num_features' : 9,
        'epochs' : 50,
        'batch_size' : 227,
        'num_cnn_layers' : 2, # Number of additional layers
        'num_gru_layers' : 2, # Number of additional layers
        'optimizer' : 'sgd',
        'dropout' : 0.015,
        'lookback' : 718,
        'activation_cnn' : 'relu',
        'activation_gru' : 'tanh',
        'cnn_layer_size_1' : 70,
        'cnn_layer_size_2' : 106,
        'gru_layer_size_1' : 51,
        'gru_layer_size_2' : 194,
        'learning_rate' : 0.024,
        'momentum' : 0.9,
        'dataset' : 'kolding'
    }
    return wandb_config
    

def run_tuner():
    
    wandb.init(config=log_defaults(), group='cnnrnn-sweep-2', project='thesis')
    
    model = build_model(config=wandb.config)
    
    train, val, _ = load_data(config=wandb.config)

    callbacks = [WandbCallback()]
    
    model.fit(
        train,
        epochs=wandb.config.epochs,
        validation_data=val,
        callbacks=callbacks
    )

In [6]:
# Sweep Config
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 5
    },
    'parameters': {
        'num_features' : {
            'value' : 9
        },
        'batch_size': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(200),
            'max': math.log(256)
        },
        'lookback': {
            'value': 24*31
        },
        'optimizer': {
            'value': 'sgd'
        },
        'dropout': {
            'distribution': 'uniform',
            'min': 0.01,
            'max': 0.5
        },
        'epochs': {
            'value': 10
        },
        'activation_gru': {
            'value': 'tanh'
        },
        'activation_cnn': {
            'value': 'relu'
        },
        'cnn_layer_size_1': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(62),
            'max': math.log(78)
        },
        'cnn_layer_size_2': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(100),
            'max': math.log(112)
        },
        'gru_layer_size_1': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(70)
        },
        'gru_layer_size_2': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(188),
            'max': math.log(200)
        },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0.001,
            'max': 0.04
        },
        'momentum': {
            'distribution': 'uniform',
            'min': 0.2,
            'max': 0.99
        },
        'num_cnn_layers': {
            'values': [0, 1, 2]
        },
        'num_gru_layers': {
            'values': [0, 1, 2]
        },
        'dataset': {
            'values': ['kolding', 'strib']
        }
    }
}

In [7]:
# Additional Parameters
split = 0.80 # split percentage for training data
steps = 1 # timesteps: 1 hour
horizon = 1 # the target hour in the future we want to predict 1 hour ahead


In [8]:
sweep_id = wandb.sweep(sweep_config, project='thesis')

Create sweep with ID: 359c75sk
Sweep URL: https://wandb.ai/nbvanting/thesis/sweeps/359c75sk


In [ ]:
wandb.agent(sweep_id, function=run_tuner, count=30)

wandb: Agent Starting Run: 8x8vt07z with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 250
wandb: 	cnn_layer_size_1: 72
wandb: 	cnn_layer_size_2: 104
wandb: 	dataset: kolding
wandb: 	dropout: 0.018265304838042416
wandb: 	epochs: 10
wandb: 	gru_layer_size_1: 62
wandb: 	gru_layer_size_2: 191
wandb: 	learning_rate: 0.0035326769839099147
wandb: 	lookback: 744
wandb: 	momentum: 0.8522771463899708
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep
c:\users\nicol\envs\thesis\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip in

Epoch 1/10
59/59 [==============================] - 68s 1s/step - loss: 0.8513 - mae: 0.7190 - mape: 385.3535 - val_loss: 1.3881 - val_mae: 0.9590 - val_mape: 301.0055
Epoch 2/10
59/59 [==============================] - 64s 1s/step - loss: 0.7111 - mae: 0.6666 - mape: 440.1818 - val_loss: 1.3761 - val_mae: 0.9502 - val_mape: 290.9503
Epoch 3/10
59/59 [==============================] - 73s 1s/step - loss: 0.7028 - mae: 0.6642 - mape: 469.8456 - val_loss: 1.3765 - val_mae: 0.9568 - val_mape: 297.7268
Epoch 4/10
59/59 [==============================] - 73s 1s/step - loss: 0.6992 - mae: 0.6607 - mape: 369.9667 - val_loss: 1.3641 - val_mae: 0.9497 - val_mape: 290.0808
Epoch 5/10
59/59 [==============================] - 68s 1s/step - loss: 0.6856 - mae: 0.6537 - mape: 678.8828 - val_loss: 1.3591 - val_mae: 0.9541 - val_mape: 295.1304
Epoch 6/10
59/59 [==============================] - 67s 1s/step - loss: 0.6844 - mae: 0.6563 - mape: 389.4756 - val_loss: 1.3567 - val_mae: 0.9623 - val_mape: 3

epoch,9
loss,0.65274
mae,0.64521
mape,521.67957
val_loss,1.23259
val_mae,0.90062
val_mape,249.47147
_runtime,698
_timestamp,1620128221
_step,9
best_val_loss,1.23259


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▄▃▃▃▂▂▁▁
mae,█▄▃▃▃▂▂▂▁▁
mape,▁▃▃▄▆▅▅▄▆█
val_loss,█▇▇▇▇▇▅▃▄▁
val_mae,█▇▇▇▇█▄▂▇▁
val_mape,█▆▇▆▇█▄▁█▂
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: 9w92np4b with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 256
wandb: 	cnn_layer_size_1: 72
wandb: 	cnn_layer_size_2: 104
wandb: 	dataset: strib
wandb: 	dropout: 0.3562743156002092
wandb: 	epochs: 10
wandb: 	gru_layer_size_1: 50
wandb: 	gru_layer_size_2: 196
wandb: 	learning_rate: 0.010370659787089297
wandb: 	lookback: 744
wandb: 	momentum: 0.5220828738785559
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

Epoch 1/10
73/73 [==============================] - 276s 4s/step - loss: 0.7163 - mae: 0.6628 - mape: 356.6735 - val_loss: 0.8881 - val_mae: 0.7326 - val_mape: 263.6144
Epoch 2/10
73/73 [==============================] - 269s 4s/step - loss: 0.5241 - mae: 0.5561 - mape: 419.4080 - val_loss: 0.8056 - val_mae: 0.6965 - val_mape: 280.7363
Epoch 3/10
73/73 [==============================] - 267s 4s/step - loss: 0.4573 - mae: 0.5189 - mape: 404.7264 - val_loss: 0.6988 - val_mae: 0.6359 - val_mape: 290.5775
Epoch 4/10
73/73 [==============================] - 267s 4s/step - loss: 0.4103 - mae: 0.4893 - mape: 334.2630 - val_loss: 0.6121 - val_mae: 0.5791 - val_mape: 238.3814
Epoch 5/10
73/73 [==============================] - 268s 4s/step - loss: 0.3782 - mae: 0.4655 - mape: 280.2192 - val_loss: 0.5844 - val_mae: 0.5639 - val_mape: 231.4915
Epoch 6/10
73/73 [==============================] - 266s 4s/step - loss: 0.3448 - mae: 0.4413 - mape: 485.5671 - val_loss: 0.5583 - val_mae: 0.5278 - val_m

epoch,9
loss,0.25818
mae,0.38305
mape,335.85718
val_loss,0.39608
val_mae,0.4443
val_mape,159.1329
_runtime,2696
_timestamp,1620132910
_step,9
best_val_loss,0.39608


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▄▄▃▂▂▂▁▁
mae,█▆▅▄▃▂▂▂▁▁
mape,▃▅█▆▁▇▄▇▃▂
val_loss,█▇▅▄▄▃▃▂▁▁
val_mae,█▇▆▄▄▃▄▂▂▁
val_mape,▇▇█▅▅▃▅▂▂▁
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Sweep Agent: Waiting for job.
500 response executing GraphQL.
{"errors":[{"message":"Post \"http://anaconda.default.svc.cluster.local/search\": context deadline exceeded","path":["agentHeartbeat"]}],"data":{"agentHeartbeat":null}}
wandb: ERROR Error while calling W&B API: Post "http://anaconda.default.svc.cluster.local/search": context deadline exceeded (<Response [500]>)
wandb: Job received.
wandb: Agent Starting Run: i6nfubcq with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 219
wandb: 	cnn_layer_size_1: 69
wandb: 	cnn_layer_size_2: 101
wandb: 	dataset: kolding
wandb: 	dropout: 0.4599103820888477
wandb: 	epochs: 10
wandb: 	gru_layer_size_1: 45
wandb: 	gru_layer_size_2: 197
wandb: 	learning_rate: 0.024923394424840387
wandb: 	lookback: 744
wandb: 	momentum: 0.7754471115694828
wandb: 	num_cnn_layers: 0
wandb: 	num_features: 9
wandb: 	num_gru_layers: 1
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a swe

Epoch 1/10
67/67 [==============================] - 303s 4s/step - loss: 0.7529 - mae: 0.6782 - mape: 385.8344 - val_loss: 0.8832 - val_mae: 0.7144 - val_mape: 225.2812
Epoch 2/10
67/67 [==============================] - 308s 5s/step - loss: 0.5533 - mae: 0.5705 - mape: 465.9402 - val_loss: 0.8587 - val_mae: 0.7409 - val_mape: 327.1571
Epoch 3/10
67/67 [==============================] - 289s 4s/step - loss: 0.5033 - mae: 0.5419 - mape: 301.5889 - val_loss: 0.6812 - val_mae: 0.6164 - val_mape: 236.7942
Epoch 4/10
67/67 [==============================] - 293s 4s/step - loss: 0.4221 - mae: 0.4950 - mape: 540.2770 - val_loss: 0.6063 - val_mae: 0.5774 - val_mape: 220.1642
Epoch 5/10
67/67 [==============================] - 295s 4s/step - loss: 0.3860 - mae: 0.4694 - mape: 918.5082 - val_loss: 0.5629 - val_mae: 0.5627 - val_mape: 216.8304
Epoch 6/10
67/67 [==============================] - 300s 4s/step - loss: 0.3567 - mae: 0.4462 - mape: 438.8760 - val_loss: 0.5157 - val_mae: 0.4907 - val_m